## 'Tiny' Resnet 3D Model

## 1. Libraries

In [1]:
#########################################################################
# 01. Libraries

import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow.keras import layers, models, optimizers, regularizers, constraints, initializers
physical_devices = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')
    
#########################################################################

## 2. Model

In [2]:
#########################################################################
# 02. Model Functions


class AttentionModel(models.Model):
    """Instantiates attention model.
    Uses two [kernel_size x kernel_size] convolutions and softplus as activation
    to compute an attention map with the same resolution as the featuremap.
    Features l2-normalized and aggregated using attention probabilites as weights.
    """

    def __init__(self, feature_dim, kernel_size=1, decay=1e-4, max_norm=0.1, name='attention'):
        """Initialization of attention model.
        Args:
          kernel_size: int, kernel size of convolutions.
          decay: float, decay for l2 regularization of kernel weights.
          name: str, name to identify model.
        """
        super(AttentionModel, self).__init__(name=name)

        # First convolutional layer (called with relu activation).
        self.conv1 = layers.Conv3D(
            feature_dim,
            (1, kernel_size, kernel_size),
            kernel_regularizer=regularizers.l2(decay),
            # kernel_constraint=constraints.MaxNorm(max_norm),
            padding='same',
            name='attn_conv1')
        self.bn_conv1 = layers.BatchNormalization(axis=-1, name='bn_conv1')
        self.relu = layers.Activation('relu')
        
        # Second convolutional layer, with softplus activation.
        self.conv2 = layers.Conv3D(
            1,
            (1, kernel_size, kernel_size),
            kernel_regularizer=regularizers.l2(decay),
            # kernel_constraint=constraints.MaxNorm(max_norm),
            padding='same',
            name='attn_conv2')


    def call(self, inputs, training=True):
        x = self.conv1(inputs)
        x = self.bn_conv1(x, training=training)
        x = self.relu(x)

        score = self.conv2(x)
        prob = tf.nn.softplus(score)

        # L2-normalize the featuremap before pooling.
        inputs = tf.nn.l2_normalize(inputs, axis=-1)
        feat = tf.reduce_mean(tf.multiply(inputs, prob), [1, 2, 3], keepdims=False)

        return feat, prob, score
    

class CustomFibrosisImageModel(models.Model):
    
    def __init__(self, kernel_attention=1, max_norm=0.1):
        super(CustomFibrosisImageModel, self).__init__(name='CustomFibrosisImageModel')
        
        self.input_avg_pool = layers.AvgPool3D(pool_size=(2, 1, 1))
        
        self.input_batch_norm = layers.BatchNormalization(axis=-1)
        
        self.block0_conv1 = layers.Conv3D(8, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_0_bn1 = layers.BatchNormalization(axis=-1)
        self.block0_conv2 = layers.Conv3D(8, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_0_bn2 = layers.BatchNormalization(axis=-1)
        self.block0_conv3 = layers.Conv3D(8, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block0_max_pool = layers.MaxPool3D(pool_size=(1, 2, 2), strides=(1, 2, 2), data_format='channels_last')
        
        self.block1_dropout = layers.Dropout(0.3)
        self.block1_conv1 = layers.Conv3D(16, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_1_bn1 = layers.BatchNormalization(axis=-1)
        self.block1_conv2 = layers.Conv3D(16, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_1_bn2 = layers.BatchNormalization(axis=-1)
        self.block1_max_pool = layers.MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), data_format='channels_last')
        
        self.block2_dropout = layers.Dropout(0.3)
        self.block2_conv1 = layers.Conv3D(32, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_2_bn1 = layers.BatchNormalization(axis=-1)
        self.block2_conv2 = layers.Conv3D(32, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           padding='same')
        self.block_2_bn2 = layers.BatchNormalization(axis=-1)
        self.block2_max_pool = layers.MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), data_format='channels_last')
        
        self.block3_dropout = layers.Dropout(0.3)
        self.block3_conv1 = layers.Conv3D(64, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_3_bn1 = layers.BatchNormalization(axis=-1)
        self.block3_conv2 = layers.Conv3D(64, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_3_bn2 = layers.BatchNormalization(axis=-1)
        self.block3_max_pool = layers.MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), data_format='channels_last')
        
        self.block4_dropout = layers.Dropout(0.4)
        self.block4_conv1 = layers.Conv3D(128, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_4_bn1 = layers.BatchNormalization(axis=-1)
        self.block4_conv2 = layers.Conv3D(128, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_4_bn2 = layers.BatchNormalization(axis=-1)
        self.block4_max_pool = layers.MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2),data_format='channels_last')
        
        self.block5_dropout = layers.Dropout(0.4)
        self.block5_conv1 = layers.Conv3D(256, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_5_bn1 = layers.BatchNormalization(axis=-1)
        self.block5_conv2 = layers.Conv3D(256, kernel_size=(1, 3, 3), activation=None,
                                           kernel_regularizer=regularizers.l2(1e-4),
                                           kernel_initializer = initializers.RandomNormal(stddev=0.01),
                                           padding='same')
        self.block_5_bn2 = layers.BatchNormalization(axis=-1)

        ##############
        
        self.att_0 = AttentionModel(8, kernel_size=kernel_attention, decay=1e-4, max_norm=max_norm, name='att0')
        self.att_1 = AttentionModel(16, kernel_size=kernel_attention, decay=1e-4, max_norm=max_norm, name='att1')
        self.att_2 = AttentionModel(32, kernel_size=kernel_attention, decay=1e-4, max_norm=max_norm, name='att2')
        self.att_3 = AttentionModel(64, kernel_size=kernel_attention, decay=1e-4, max_norm=max_norm, name='att3')
        self.att_4 = AttentionModel(128, kernel_size=kernel_attention, decay=1e-4, max_norm=max_norm, name='att4')
        self.att_5 = AttentionModel(256, kernel_size=kernel_attention, decay=1e-4, max_norm=max_norm, name='att4')
        
        #############
        
        
        
    def call(self, inputs, training=True):
        x =  self.input_avg_pool(inputs)
        
        x = self.input_batch_norm(x, training)
        
        x = self.block0_conv1(x)
        block0_stack0 = x
        x = self.block_0_bn1(x, training)
        x = tf.nn.relu(x)
        x = self.block0_conv2(x)
        x = self.block_0_bn2(x, training)
        x = tf.nn.relu(x)
        x = self.block0_conv3(x)
        block0_stack1 = x  
        x = tf.add(block0_stack0, block0_stack1)
#         att_0_feat, att_0_prob, _ = self.att_0(x, training)
        x = self.block0_max_pool(x)    
       
        x = self.block1_dropout(x, training)
        x = self.block1_conv1(x)
        block1_stack0 = x
        x = self.block_1_bn1(x, training)
        x = tf.nn.relu(x)
        x = self.block1_conv2(x)
        block1_stack1 = x
        x = tf.add(block1_stack0, block1_stack1)
        x = self.block_1_bn2(x, training)
        x = tf.nn.relu(x)
        att_1_feat, att_1_prob, _ = self.att_1(x, training)
        x = self.block1_max_pool(att_1_prob)      
        
        x = self.block2_dropout(x, training)
        x = self.block2_conv1(x)
        block2_stack0 = x
        x = self.block_2_bn1(x, training)
        x = tf.nn.relu(x)
        x = self.block2_conv2(x) 
        block2_stack1 = x
        x = tf.add(block2_stack0, block2_stack1)
        x = self.block_2_bn2(x, training)
        x = tf.nn.relu(x)
        att_2_feat, att_2_prob, _ = self.att_2(x, training)
        x = self.block2_max_pool(att_2_prob)        
        
        x = self.block3_dropout(x, training)
        x = self.block3_conv1(x)
        block3_stack0 = x
        x = self.block_3_bn1(x, training)
        x = tf.nn.relu(x)
        x = self.block3_conv2(x)
        block3_stack1 = x
        x = tf.add(block3_stack0, block3_stack1)
        x = self.block_3_bn2(x, training)
        x = tf.nn.relu(x)
        att_3_feat, att_3_prob, _ = self.att_3(x, training)
        x = self.block3_max_pool(att_3_prob)
        
        x = self.block4_dropout(x, training)
        x = self.block4_conv1(x)
        block4_stack0 = x
        x = self.block_4_bn1(x, training)
        x = tf.nn.relu(x)
        x = self.block4_conv2(x)
        block4_stack1 = x
        x = tf.add(block4_stack0, block4_stack1)
        x = self.block_4_bn2(x, training)
        x = tf.nn.relu(x)
        att_4_feat, att_4_prob, _ = self.att_4(x, training)
        x = self.block4_max_pool(att_4_prob)
        
        x = self.block5_dropout(x, training)
        x = self.block5_conv1(x)
        block5_stack0 = x
        x = self.block_5_bn1(x, training)
        x = tf.nn.relu(x)
        x = self.block5_conv2(x)
        block5_stack1 = x
        x = tf.add(block5_stack0, block5_stack1)
        x = self.block_5_bn2(x, training)
        x = tf.nn.relu(x)
        att_5_feat, att_5_prob, _ = self.att_5(x, training)
        
        features_vector = tf.concat([att_1_feat,
                                     att_2_feat,
                                     att_3_feat,
                                     att_4_feat,
                                     att_5_feat], axis=-1)
                
        return features_vector
          

#########################################################################

In [3]:
# p = tf.random.normal((1, 32, 160, 160, 1))
# m = CustomFibrosisImageModel()
# # tf.nn.avg_pool3d(p, (2, 1, 1), strides=1, padding='SAME')

# m(p, training=True)

In [4]:
#########################################################################

model = CustomFibrosisImageModel(kernel_attention=1, max_norm=0.1)
p = tf.random.normal((1, 32, 220, 220, 1))
model(p)
# m.build(input_shape=(None, None, None, None, 1))
# input_shape=(None, None, None, 1)
# input_ = layers.Input(shape=input_shape)
# model = models.Model(inputs=input_, outputs=m.output, name='Resnet3D')

# print(model.summary(line_length=130))

tf.keras.models.save_model(model=model,
                           save_format='tf',
                           filepath='../05_Saved_Models/customModel', 
                           include_optimizer=False)

#########################################################################

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../05_Saved_Models/customModel\assets
